In [3]:
def get_request(q):
    return(spotify.search(q, limit=1, offset=0, type='track', market="GB")['tracks']['items'][0])
    
def get_track_info(artistName, trackName):
    original_artistName = artistName
    original_trackName = trackName
    i = 1
    while i<=3:
        try:
            q = "artist:" + artistName + " track:" + trackName
            data = get_request(q)
            break
        except:
            if i == 1:
                artistName = re.sub("(\[.*\])|(\(.*\))|(\*.*\*)",  '',    artistName).strip()  
                trackName = re.sub("(\[.*\])|(\(.*\))|(\*.*\*)",  '',    trackName).strip()
            if i == 2:
                artistName = re.sub("[',’]",  '',    artistName).strip()  
                trackName = re.sub("[',’]",  '',    trackName).strip()
            i += 1
            
    if i == 4:
        print('Error with: ', artistName, ":", trackName)
        return(pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame())
    
    return(clean_track_data(data, 
                            search = True, 
                            original_artistName = original_artistName,
                            original_trackName = original_trackName,
                            artistName = artistName,
                            trackName = trackName,
                            q = q))

def clean_track_data(data, 
                     search = False, 
                     original_artistName = None,
                     original_trackName = None,
                     artistName = None,
                     trackName = None,
                     q = None):
    artist_data = data['artists']
    album_data = data['album']
    album_artists = album_data['artists']
    data.pop('album',None)
    data.pop('artists',None)

    artist_data_df = pd.concat([pd.DataFrame(artist_data),pd.DataFrame(album_artists)])
    artist_data_df.rename(columns = {'external_urls': "Artist External URL's",
                        'href': "Artist href",
                        'id': "Artist ID",
                        'name': "Artist Name",
                        'type': "Artist Type",
                        'uri': "Artist URI"}, inplace=True)
    artist_data_df = artist_data_df.drop_duplicates(subset='Artist ID', keep='first')

    album_data['artists'] = [album_artist['id'] for album_artist in album_data['artists']]
    album_data_df = pd.DataFrame({k: [v] for k, v in album_data.items()})
    album_data_df.rename(columns={'album_type': "Album Type", 
                            'artists': "Album Artists", 
                            'external_urls': "External URL's", 
                            'href': "href", 
                            'id': "Album ID", 
                            'images': "Album Images",
                            'name': "Album Name", 
                            'release_date': "Album Release Date", 
                            'release_date_precision': "Album Release Date Precision", 
                            'total_tracks': "Album Total Tracks",
                            'type': "Album Object Type", 
                            'uri': "Album URI"}, inplace=True)
    album_data_df.drop(labels='available_markets', axis=1, inplace=True, errors='ignore')
    
    data['Album ID'] = album_data['id']
    data["Artist ID's"] = [artist['id'] for artist in artist_data]
    track_data_df = pd.DataFrame({k: [v] for k, v in data.items()})
    track_data_df.rename(columns={'disc_number': 'Disc Number', 
                            'duration_ms': 'Track Duration (ms)', 
                            'explicit': 'Is Track Explicit', 
                            'external_ids': "Track External ID's",
                            'external_urls': "Track External URL's",
                            'href': 'Track href', 
                            'id': 'Track ID', 
                            'is_local': 'Is Track Local', 
                            'is_playable': 'Is Track Playable', 
                            'name': 'Track Name',
                            'popularity': 'Track Popularity', 
                            'preview_url': 'Track Preview URL', 
                            'track_number': 'Track Number', 
                            'type': 'Track Type', 
                            'uri': 'Track URI'}, inplace=True)
    track_data_df.drop(labels=['available_markets','episode','track'], axis=1, inplace=True, errors='ignore')
    
    search_data_df = pd.DataFrame()
    if search == True:
        search_data_df['Original Search Artist'] = original_artistName
        search_data_df['Original Search Track'] = original_trackName
        search_data_df['Cleaned Search Artist'] = artistName
        search_data_df['Cleaned Search Track'] = trackName
        search_data_df['Search Query'] = q
        search_data_df['Track ID'] = track_data_df['Track ID']
    
    return(track_data_df,album_data_df,artist_data_df,search_data_df)

In [ ]:
def make_playlist(name,description, public = False, collaborative=False):
    playlist = spotify.user_playlist_create(my_id, name, public=public, collaborative=collaborative, description=description)
    playlist.pop('owner',None)
    playlist.pop('tracks',None)
    playlist = pd.DataFrame({k: [v] for k, v in playlist.items()})
    global playlists
    playlists = pd.concat([playlist, playlists])
    return(playlists)

def edit_playlist_details(playlist_id, user = my_id, name=None, public=None, collaborative=None, description=None):
    spotify.user_playlist_change_details(user, playlist_id, name=name, public=public, collaborative=collaborative, description=description)
    global playlists
    if name != None:
        playlists.loc[playlists['id'] == playlist_id,['name']] = name
    if description != None:
        playlists.loc[playlists['id'] == playlist_id,['description']] = description
    if collaborative != None:
        playlists.loc[playlists['id'] == playlist_id,['collaborative']] = collaborative
    if public != None:
        playlists.loc[playlists['id'] == playlist_id,['public']] = public
    return(playlists)

def delete_playlist(playlist_id, user = my_id):
    spotify.user_playlist_unfollow(user, playlist_id)
    global playlists
    playlists = playlists[~(playlists['id'] == playlist_id)]
    return(playlists)

def get_playlist(playlist_id, with_tracks = True):
    playlist = spotify.playlist(playlist_id)
    playlist.pop('owner',None)
    playlist.pop('tracks',None)
    global albums, artists, playlists, tracks, playlist_tracks
    playlist = pd.DataFrame({k: [v] for k, v in playlist.items()})
    playlists = pd.concat([playlists, playlist]).drop_duplicates(subset = 'id', keep = "last")
    
    if with_tracks == True:
        offset = 0
        total = 10000
        temp = []
        while offset<total:
            track_data = spotify.user_playlist_tracks(user=my_id,playlist_id=playlist_id, offset = offset)
            temp = temp + [clean_track_data(x['track']) for x in track_data['items']]
            if offset == 0:
                total = track_data['total']
            offset += 100
        if temp != []:
            tracks_instance = pd.concat([item[0] for item in temp])
            tracks = pd.concat([tracks, tracks_instance]).drop_duplicates(subset='Track ID', keep='last')
            tracks_instance['Playlist ID'] = playlist_id
            tracks_instance['Position Number'] = np.arange(len(tracks_instance))
        
            if playlist_tracks.size != 0:
                playlist_tracks = pd.concat([playlist_tracks[playlist_tracks['Playlist ID'] != playlist_id], tracks_instance[['Track ID', 'Playlist ID', 'Position Number']]])
            else:
                playlist_tracks = tracks_instance[['Track ID', 'Playlist ID', 'Position Number']]
            albums = pd.concat([albums] + [item[1] for item in temp]).drop_duplicates(subset='Album ID', keep='last')
            artists = pd.concat([artists] + [item[2] for item in temp]).drop_duplicates(subset='Artist ID', keep='last')
        
        return(playlist_tracks)
    
    else:
        return(playlists)

In [ ]:
def edit_playlist_tracks(playlist_id, new_tracks):
    
    get_playlist(playlist_id = playlist_id, with_tracks = True)

    new_tracks_df = pd.DataFrame()
    new_tracks_df['Track ID'] = pd.Series(new_tracks)
    new_tracks_df['New Position Number'] = np.arange(len(new_tracks))
    
    if playlist_tracks.size != 0:
        current_playlist = playlist_tracks[playlist_tracks['Playlist ID'] == playlist_id]
        new_tracks_df = new_tracks_df.merge(current_playlist, how = 'outer', on = 'Track ID')
    else:
        current_playlist = playlist_tracks
        new_tracks_df['Position Number'] = pd.NA
        new_tracks_df['Playlist ID'] = pd.NA 
    
    remove_tracks = list(new_tracks_df[new_tracks_df['New Position Number'].isna()]['Track ID'])
    if len(remove_tracks) != 0:
        spotify.playlist_remove_all_occurrences_of_items(playlist_id = playlist_id, items = remove_tracks)
    
    new_tracks_df = new_tracks_df[~new_tracks_df['New Position Number'].isna()].sort_values(by = 'Position Number')
    
    add_tracks = list(new_tracks_df[new_tracks_df['Position Number'].isna()]['Track ID'])
    if len(add_tracks) != 0:
        i = 0
        while i < len(add_tracks) // 50 + 1:
            temp_tracks_to_add = add_tracks[50*i:50*(i+1)]
            spotify.playlist_add_items(playlist_id = playlist_id, items = temp_tracks_to_add, position=50*i)
            i += 1
        
    new_tracks_df['Position Number'] = np.arange(len(new_tracks))
    new_tracks_df = new_tracks_df.sort_values(by = 'New Position Number')

    for i in range(len(new_tracks_df.index)):
        new_pos = int(new_tracks_df.loc[i, 'New Position Number'])
        pos = int(new_tracks_df.loc[i, 'Position Number'])
        if new_pos != pos:
            spotify.playlist_reorder_items(playlist_id = playlist_id, range_start=pos, insert_before=new_pos, range_length=1)
            new_tracks_df.loc[(new_tracks_df['Position Number'] < pos) & (new_tracks_df['Position Number'] >= new_pos),'Position Number'] += 1
            new_tracks_df.loc[(new_tracks_df['New Position Number'] == new_pos),'Position Number'] = new_pos
    
    get_playlist(playlist_id = playlist_id, with_tracks = True)
    current_playlist = playlist_tracks[playlist_tracks['Playlist ID'] == playlist_id]
    
    return(current_playlist)